In [ ]:
import pandas as pd
import numpy as np

In [ ]:

df = pd.read_html('https://fbref.com/en/squads/b8fd03ef/2013-2014/matchlogs/c9/schedule/Manchester-City-Scores-and-Fixtures-Premier-League', attrs={'id':'matchlogs_for'})[0] # don't forget to array index #
df.drop(columns=['Time','Day','Poss','Attendance','Captain','Formation','Opp Formation', 'Referee', 'Match Report', 'Notes'], inplace=True, errors='ignore')

In [ ]:
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date']) # Standardise the date #
# 1) Give matchday a discrete value only
df['matchday'] = df['Round'].str.extract(r'(\d+)').astype(int) # extract the integer value from the matchday description #
# 2) Map result to a value
df['points_won'] = df['Result'].map({'W':3,'D':1,'L':0})
# 3) Normalise the venue to H/A binary values
df['home_0_away_1'] = df['Venue'].map({'Home':0,'Away':1})
df.drop(columns=['Round'], inplace=True, errors='ignore')

In [ ]:
df.drop(columns=['Venue'], inplace=True, errors='ignore')

In [ ]:
df_elo = pd.read_csv('/content/EloRatings.csv')
df_elo['date'] = pd.to_datetime(df_elo['date'], dayfirst=True)

In [ ]:
# We will have to filter out elo clubs as well #
#1) Limit it to England #
df_elo = df_elo[df_elo['country']=='ENG']
# 2) Restrict the timeframe of the df #
cutoff = pd.Timestamp('2013-07-01')
cutoff_2 = pd.Timestamp('2015-07-01')
df_elo = df_elo[(df_elo['date'] >= cutoff) & (df_elo['date'] <= cutoff_2)].reset_index(drop=True)


Make sure that names in the dataframe are on the left (ones to be mapped onto) while names we have to map to df are on the right. Not the other way round.

In [ ]:
def round_elo_to_date(dt):
  return pd.Timestamp(dt.year, dt.month,1) if dt.day<15 else pd.Timestamp(dt.year, dt.month,15) # If day less than 15 return for 1st else return for 15th #
df['merge_date'] = df['Date'].apply(round_elo_to_date)
df_elo['merge_date'] = df_elo['date']
name_map = {
    'Arsenal': 'Arsenal',
    'Aston Villa': 'Aston Villa',
    'Cardiff City': 'Cardiff',
    'Chelsea': 'Chelsea',
    'Crystal Palace': 'Crystal Palace',
    'Everton': 'Everton',
    'Fulham': 'Fulham',
    'Hull City': 'Hull',
    'Liverpool': 'Liverpool',
    'Manchester Utd': 'Man United',
    'Newcastle Utd': 'Newcastle',
    'Norwich City': 'Norwich',
    'Southampton': 'Southampton',
    'Stoke City': 'Stoke',
    'Sunderland': 'Sunderland',
    'Swansea City': 'Swansea',
    'Tottenham': 'Tottenham',
    'West Brom': 'West Brom',
    'West Ham': 'West Ham',
}
df['elo_opponent'] = df['Opponent'].map(name_map)
df['elo_club'] = 'Man City'
df = df.merge(
    df_elo[['club', 'merge_date', 'elo']].rename(columns={'elo': 'opp_elo'}),
    left_on=['elo_opponent', 'merge_date'],
    right_on=['club', 'merge_date'],
    how='left'
).drop(columns='club')
df = df.merge(
    df_elo[['club', 'merge_date', 'elo']].rename(columns={'elo': 'city_elo'}),
    left_on=['elo_club', 'merge_date'],
    right_on=['club', 'merge_date'],
    how='left'
).drop(columns=['club', 'elo_club','merge_date','elo_opponent'])

In [ ]:
df

,Date,Result,GF,GA,Opponent,matchday,points_won,home_0_away_1,opp_elo,city_elo
0,2013-08-19,W,4,0,Newcastle Utd,1,3,0,1664.19,1867.47
1,2013-08-25,L,2,3,Cardiff City,2,0,1,1570.36,1867.47
2,2013-08-31,W,2,0,Hull City,3,3,0,1510.86,1867.47
3,2013-09-14,D,0,0,Stoke City,4,1,1,1660.96,1866.28
4,2013-09-22,W,4,1,Manchester Utd,5,3,0,1911.61,1866.28
5,2013-09-28,L,2,3,Aston Villa,6,0,1,1671.77,1866.28
6,2013-10-05,W,3,1,Everton,7,3,0,1814.60,1884.02
7,2013-10-19,W,3,1,West Ham,8,3,1,1687.92,1889.48
8,2013-10-27,L,1,2,Chelsea,9,0,1,1907.57,1889.48
9,2013-11-02,W,7,0,Norwich City,10,3,0,1655.51,1896.48


In [ ]:
df['elo_diff'] = df['city_elo'] - df['opp_elo']
df['days_since_last_game_city'] = df['Date'].diff().dt.days.fillna(0).astype(int)

In [ ]:
df.drop(columns=['days_since_last_city'], inplace=True, errors='ignore')

In [ ]:
position_map = {
    'Manchester City': 1,
    'Liverpool': 2,
    'Chelsea': 3,
    'Arsenal': 4,
    'Everton': 5,
    'Tottenham': 6,
    'Manchester Utd': 7,
    'Southampton': 8,
    'Stoke City': 9,
    'Newcastle Utd': 10,
    'Crystal Palace': 11,
    'Swansea City': 12,
    'West Ham': 13,
    'Sunderland': 14,
    'Aston Villa': 15,
    'Hull City': 16,
    'West Brom': 17,
    'Norwich City': 18,
    'Fulham': 19,
    'Cardiff City': 20
}
df['opp_final_pos'] = df['Opponent'].map(position_map)

In [ ]:
# Now we will compute odds of city win using B365 #
df_bet = pd.read_csv('/content/13-14Odds.csv')
df_bet['Date'] = pd.to_datetime(df_bet['Date'], dayfirst=True)
home_odds_map = df_bet.set_index(['Date','HomeTeam'])['B365H'].to_dict()
away_odds_map = df_bet.set_index(['Date','AwayTeam'])['B365A'].to_dict()
def get_city_win_odds(row):
    key = (row['Date'], 'Man City')
    if row['home_0_away_1'] == 0:
        return home_odds_map.get(key, pd.NA) # Get Home win odds #
    else:
        return away_odds_map.get(key, pd.NA) # Get away win odds #

df['city_win_odds'] = df.apply(get_city_win_odds, axis=1)

In [ ]:
df

In [ ]:
df['days_since_last_game_opp'] = [0, 8, 7, 14, 8, 7, 5, 13, 8, 7, 8, 14, 8, 4, 3, 6, 7, 5, 2,
 4, 11, 7, 10, 5, 7, 10, 14, 7, 3, 4, 7, 7, 4, 9, 8, 7, 4, 8]
df

In [ ]:
df = df[['matchday','Date','home_0_away_1','Opponent','Result','points_won','GF','GA','city_elo', 'opp_elo', 'elo_diff', 'city_win_odds', 'opp_final_pos', 'days_since_last_game_city', 'days_since_last_game_opp']]

In [ ]:
# Final df for 13/14 season #
df

In [ ]:
df_2 = pd.read_html('https://fbref.com/en/squads/b8fd03ef/2014-2015/matchlogs/c9/schedule/Manchester-City-Scores-and-Fixtures-Premier-League', attrs = {'id':'matchlogs_for'})[0]
df_2.drop(columns=['Time','Day','Poss','Attendance','Captain','Formation','Opp Formation', 'Referee', 'Match Report', 'Notes'], inplace=True, errors='ignore')

In [ ]:
df_2

In [ ]:
df_2['Date'] = pd.to_datetime(df_2['Date']) # Standardise the date #
# 1) Give matchday a discrete value only
df_2['matchday'] = df_2['Round'].str.extract(r'(\d+)').astype(int) # extract the integer value from the matchday description #
# 2) Map result to a value
df_2['points_won'] = df_2['Result'].map({'W':3,'D':1,'L':0})
# 3) Normalise the venue to H/A binary values
df_2['home_0_away_1'] = df_2['Venue'].map({'Home':0,'Away':1})
df_2.drop(columns=['Round'], inplace=True, errors='ignore')

In [ ]:
df_2.drop(columns=['Venue'], inplace=True, errors='ignore')

In [ ]:
def round_elo_to_date(dt):
  return pd.Timestamp(dt.year, dt.month,1) if dt.day<15 else pd.Timestamp(dt.year, dt.month,15) # If day less than 15 return for 1st else return for 15th #
df_2['merge_date'] = df_2['Date'].apply(round_elo_to_date)
df_elo['merge_date'] = df_elo['date']
name_map_2 = {
    'Arsenal':           'Arsenal',
    'Aston Villa':       'Aston Villa',
    'Burnley':           'Burnley',
    'Chelsea':           'Chelsea',
    'Crystal Palace':    'Crystal Palace',
    'Everton':           'Everton',
    'Hull City':         'Hull',
    'Leicester City':    'Leicester',
    'Liverpool':         'Liverpool',
    'Manchester Utd':    'Man United',
    'Newcastle Utd':     'Newcastle',
    'QPR':               'QPR',
    'Southampton':       'Southampton',
    'Stoke City':        'Stoke',
    'Sunderland':        'Sunderland',
    'Swansea City':      'Swansea',
    'Tottenham':         'Tottenham',
    'West Brom':         'West Brom',
    'West Ham':          'West Ham',
}
df_2['elo_opponent'] = df_2['Opponent'].map(name_map_2) # Be careful with copy-paste #
df_2['elo_club'] = 'Man City'
df_2 = df_2.merge(
    df_elo[['club', 'merge_date', 'elo']].rename(columns={'elo': 'opp_elo'}),
    left_on=['elo_opponent', 'merge_date'],
    right_on=['club', 'merge_date'],
    how='left'
).drop(columns='club')
df_2 = df_2.merge(
    df_elo[['club', 'merge_date', 'elo']].rename(columns={'elo': 'city_elo'}),
    left_on=['elo_club', 'merge_date'],
    right_on=['club', 'merge_date'],
    how='left'
).drop(columns=['club', 'elo_club','merge_date','elo_opponent'])

In [ ]:
df_2.drop(columns=['opp_elo_x','city_elo_x'], inplace=True, errors='ignore')
df_2

In [ ]:
df_2['elo_diff'] = df_2['city_elo'] - df_2['opp_elo']
df_2['days_since_last_game_city'] = (
    df_2['Date']
      .diff()               # current_date - previous_date
      .dt.days              # convert to integer days
      .fillna(0)            # first match has no “previous” → 0
      .astype(int)
)
df_2.drop(columns=['days_since_last_game_united'], inplace=True, errors='ignore')
df_2

In [ ]:
print(df_2['Opponent'].unique())

['Newcastle Utd' 'Liverpool' 'Stoke City' 'Arsenal' 'Chelsea' 'Hull City'
 'Aston Villa' 'Tottenham' 'West Ham' 'Manchester Utd' 'QPR'
 'Swansea City' 'Southampton' 'Sunderland' 'Everton' 'Leicester City'
 'Crystal Palace' 'West Brom' 'Burnley']


In [ ]:
df_2['days_since_last_game_opp'] = [0, 8, 6, 13, 8, 7, 7, 14, 7, 7, 7, 13, 6, 4, 3, 6, 7, 6, 2,
 4, 9, 7, 14, 7, 3, 10, 7, 10, 10, 7, 23, 8, 8, 14, 8, 8, 6, 8]
final_pos_2 = {
    'Chelsea':              1,
    'Manchester City':      2,
    'Arsenal':              3,
    'Manchester Utd':       4,
    'Tottenham':            5,
    'Liverpool':            6,
    'Southampton':          7,
    'Swansea City':         8,
    'Stoke City':           9,
    'Crystal Palace':      10,
    'Everton':             11,
    'West Ham':            12,
    'West Brom':           13,
    'Leicester City':      14,
    'Newcastle Utd':       15,
    'Sunderland':          16,
    'Aston Villa':         17,
    'Hull City':           18,
    'Burnley':             19,
    'QPR':                 20
}
df_2['opp_final_pos'] = df_2['Opponent'].map(final_pos_2)

In [ ]:
df_2 = df_2[['matchday','Date','home_0_away_1','Opponent',	'Result',	'points_won'	,'GF'	,'GA'	,'city_elo'	,'opp_elo',	'elo_diff',	'city_win_odds'	,'opp_final_pos','days_since_last_game_city','days_since_last_game_opp']]

In [ ]:
# Now we will compute odds of city win using B365 #
df_bet_2 = pd.read_csv('/content/14-15 Odds.csv')
df_bet_2['Date'] = pd.to_datetime(df_bet_2['Date'], dayfirst=True)
home_odds_map = df_bet_2.set_index(['Date','HomeTeam'])['B365H'].to_dict()
away_odds_map = df_bet_2.set_index(['Date','AwayTeam'])['B365A'].to_dict()
def get_city_win_odds(row):
    key = (row['Date'], 'Man City')
    if row['home_0_away_1'] == 0:
        return home_odds_map.get(key, pd.NA) # Get Home win odds #
    else:
        return away_odds_map.get(key, pd.NA) # Get away win odds #

df_2['city_win_odds'] = df_2.apply(get_city_win_odds, axis=1)

In [ ]:
df_2

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,city_elo,opp_elo,elo_diff,city_win_odds,opp_final_pos,days_since_last_game_city,days_since_last_game_opp
0,1,2014-08-17,1,Newcastle Utd,W,3,2,0,1955.29,1650.72,304.57,1.67,15,0,0
1,2,2014-08-25,0,Liverpool,W,3,3,1,1955.29,1899.80,55.49,1.85,6,8,8
2,3,2014-08-30,0,Stoke City,L,0,0,1,1955.29,1698.64,256.65,1.18,9,5,6
3,4,2014-09-13,1,Arsenal,D,1,2,2,1948.35,1863.08,85.27,2.50,3,14,13
4,5,2014-09-21,0,Chelsea,D,1,1,1,1947.30,1937.29,10.01,2.38,1,8,8
5,6,2014-09-27,1,Hull City,W,3,4,2,1947.30,1569.79,377.51,1.57,18,6,7
6,7,2014-10-04,1,Aston Villa,W,3,2,0,1933.22,1631.47,301.75,1.50,17,7,7
7,8,2014-10-18,0,Tottenham,W,3,4,1,1932.54,1769.91,162.63,1.45,5,14,14
8,9,2014-10-25,1,West Ham,L,0,1,2,1932.54,1665.55,266.99,1.55,12,7,7
9,10,2014-11-02,0,Manchester Utd,W,3,1,0,1920.19,1818.42,101.77,1.91,4,8,7


In [ ]:
df["season"] = "2013/14"
df_2["season"] = "2014/15"
final_df = pd.concat([df, df_2], ignore_index=True)

In [ ]:
final_df

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,city_elo,opp_elo,elo_diff,city_win_odds,opp_final_pos,days_since_last_game_city,days_since_last_game_opp,season
0,1,2013-08-19,0,Newcastle Utd,W,3,4,0,1867.47,1664.19,203.28,1.33,10,0,0,2013/14
1,2,2013-08-25,1,Cardiff City,L,0,2,3,1867.47,1570.36,297.11,1.44,20,6,8,2013/14
2,3,2013-08-31,0,Hull City,W,3,2,0,1867.47,1510.86,356.61,1.17,16,6,7,2013/14
3,4,2013-09-14,1,Stoke City,D,1,0,0,1866.28,1660.96,205.32,1.65,9,14,14,2013/14
4,5,2013-09-22,0,Manchester Utd,W,3,4,1,1866.28,1911.61,-45.33,2.20,7,8,8,2013/14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,34,2015-04-25,0,Aston Villa,W,3,3,2,1863.25,1587.17,276.08,1.29,17,6,14,2014/15
72,35,2015-05-03,1,Tottenham,W,3,1,0,1868.83,1744.23,124.60,2.05,5,8,8,2014/15
73,36,2015-05-10,0,QPR,W,3,6,0,1868.83,1552.15,316.68,1.22,20,7,8,2014/15
74,37,2015-05-17,1,Swansea City,W,3,4,2,1878.12,1691.90,186.22,1.75,8,7,6,2014/15


In [ ]:
final_df.to_csv('city_pt1.csv',index=False)